<a href="https://colab.research.google.com/github/Matheus2343/Detec-o-de-Fake-News/blob/main/Prototipo_Detec%C3%A7%C3%A3o_de_Fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q google-genai

In [5]:
!pip install -q google-search-results

In [32]:
!pip install -q google-generativeai

In [15]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [60]:
import google.generativeai as genai
from datetime import date
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials


In [63]:
def executar_pesquisa_google(query, num_resultados=3):
    """Executa uma pesquisa no Google e retorna os resultados."""
    try:
        service = build("customsearch", "v1", developerKey=GOOGLE_SEARCH_API_KEY)
        response = service.cse().list(q=query, cx=GOOGLE_CSE_ID, num=num_resultados).execute()
        return response.get("items", [])
    except Exception as e:
        print(f"Erro ao executar a pesquisa no Google: {e}")
        return []

In [67]:
def verificar_fake_news(mensagem):
    """
    Tenta detectar se uma mensagem é fake news usando o Google Gemini 2.0 Flash e a Pesquisa Google.
Identificar potenciais sinais de desinformação ou notícias falsas com base em evidências encontradas na pesquisa.
Fornecer um relatório conciso sobre os sinais identificados e o grau de confiança na avaliação.
    """
    try:
        model = genai.GenerativeModel('gemini-2.0-flash')

        query_gemini = f"Verifique a veracidade da seguinte afirmação: '{mensagem}'. Procure por relatos de agências de notícias confiáveis, sites de fact-checking e desmentidos."

        response_gemini = model.generate_content(query_gemini)
        response_text_gemini = response_gemini.text.lower()

        print(f"\nResposta do Gemini Flash:\n{response_text_gemini}\n")

        query_google = f"verificar se é verdade: {mensagem}"
        resultados_google = executar_pesquisa_google(query_google)

        print("\nResultados da Pesquisa Google:")
        for i, resultado in enumerate(resultados_google):
            print(f"[{i+1}] Título: {resultado['title']}")
            print(f"    Link: {resultado['link']}")
            print(f"    Snippet: {resultado['snippet']}")
        print("-" * 30)

        if "notícia falsa" in response_text_gemini or "alegação falsa" in response_text_gemini or "desmentido" in response_text_gemini or "fake news" in response_text_gemini:
            return "⚠️ ALERTA: A mensagem pode ser fake news com base nas informações encontradas pelo Gemini."
        elif any("fact-checking" in r.get('snippet', '').lower() or "checagem de fatos" in r.get('snippet', '').lower() for r in resultados_google):
            return "🔎 Verificações de fatos encontradas nos resultados da pesquisa. Recomenda-se analisar os links."
        elif "verdadeira" in response_text_gemini or "confirmado" in response_text_gemini or "fato" in response_text_gemini:
            return "✅ A mensagem pode ser verdadeira com base nas informações encontradas pelo Gemini."
        elif not resultados_google:
            return "🤔 O Gemini não encontrou informações conclusivas e a Pesquisa Google não retornou resultados relevantes."
        else:
            return "❓ A resposta do Gemini não forneceu informações claras e a Pesquisa Google retornou resultados que precisam ser analisados com cautela."

    except Exception as e:
        print(f"Ocorreu um erro ao verificar a mensagem: {e}")
        return "❌ Ocorreu um erro ao verificar a mensagem."

In [68]:
if __name__ == "__main__":
    print("🤖 Detector de Fake News (Protótipo com Gemini 2.0 Flash) 🤖")
    mensagem_usuario = input("Digite a mensagem que você quer verificar: ")
    if mensagem_usuario:
        resultado = verificar_fake_news(mensagem_usuario)
        print(f"\nResultado da verificação: {resultado}")
    else:
        print("Você não digitou nenhuma mensagem.")

🤖 Detector de Fake News (Protótipo com Gemini 2.0 Flash) 🤖
Digite a mensagem que você quer verificar: 
Você não digitou nenhuma mensagem.
